### 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

# %%
# Load the enriched dataset
print("Loading enriched dataset...")
df = pd.read_csv('../data/processed/ethiopia_fi_enriched.csv')
print(f"Dataset shape: {df.shape}")

# Load insights from Task 2
key_insights = pd.read_csv('../data/processed/key_insights.csv')
modeling_hypotheses = pd.read_csv('../data/processed/modeling_hypotheses.csv')

### 2. Understand the Impact Data

In [ ]:
# Separate by record type
observations = df[df['record_type'] == 'observation'].copy()
events = df[df['record_type'] == 'event'].copy()
impact_links = df[df['record_type'] == 'impact_link'].copy()
targets = df[df['record_type'] == 'target'].copy()

print(f"Observations: {observations.shape[0]} records")
print(f"Events: {events.shape[0]} records")
print(f"Impact Links: {impact_links.shape[0]} records")

# Convert dates
observations['observation_date'] = pd.to_datetime(observations['observation_date'], errors='coerce')
events['event_date'] = pd.to_datetime(events['event_date'], errors='coerce')

# %%
# Join impact links with events to get full relationships
impact_with_events = pd.merge(
    impact_links,
    events[['record_id', 'event_name', 'event_category', 'event_date', 'description']],
    left_on='parent_id',
    right_on='record_id',
    how='left'
)

print("Impact Links with Event Details:")
display(impact_with_events.head())

# %%
# Summary of impact relationships
print("="*60)
print("IMPACT RELATIONSHIPS SUMMARY")
print("="*60)

print("\n📊 Events by Category with Impact Counts:")
event_impact_summary = impact_with_events.groupby(['event_category', 'event_name']).agg({
    'event_date': 'first',
    'related_indicator': 'count',
    'impact_direction': lambda x: ', '.join(x.unique()),
    'impact_magnitude': lambda x: ', '.join(x.dropna().unique())
}).reset_index()
event_impact_summary = event_impact_summary.rename(columns={'related_indicator': 'impact_count'})
display(event_impact_summary.sort_values(['event_category', 'event_date']))

print("\n🎯 Indicators Most Affected by Events:")
indicator_impact_summary = impact_with_events.groupby(['pillar', 'related_indicator']).agg({
    'event_name': 'count',
    'impact_direction': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'mixed',
    'impact_magnitude': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'mixed'
}).reset_index()
indicator_impact_summary = indicator_impact_summary.rename(columns={'event_name': 'event_count'})
display(indicator_impact_summary.sort_values('event_count', ascending=False))

### 3. Build Event-Indicator Association Matrix

In [ ]:
# Define key indicators for forecasting
key_indicators = [
    'ACC_OWNERSHIP',           # Primary target: Account ownership
    'ACC_MM_ACCOUNT',          # Mobile money accounts
    'USG_DIGITAL_PAYMENT',     # Primary target: Digital payment usage
    'USG_P2P_VOLUME',          # P2P transaction volume
    'INF_AGENT_DENSITY',       # Infrastructure: Agent density
    'ENB_MOBILE_INTERNET',     # Enabler: Mobile internet
    'ENB_SMARTPHONE_PEN'       # Enabler: Smartphone penetration
]

# Get unique events
unique_events = events['event_name'].unique()
print(f"Unique events: {len(unique_events)}")
print(f"Key indicators: {len(key_indicators)}")

# %%
# Initialize association matrix
association_matrix = pd.DataFrame(
    index=unique_events,
    columns=key_indicators
)

# Also create matrices for direction, magnitude, and lag
direction_matrix = pd.DataFrame(index=unique_events, columns=key_indicators)
magnitude_matrix = pd.DataFrame(index=unique_events, columns=key_indicators)
lag_matrix = pd.DataFrame(index=unique_events, columns=key_indicators)

#### 3.1 Populate Matrices from Existing Impact Links

In [ ]:
# Helper function to convert impact magnitude to numeric score
def magnitude_to_score(magnitude):
    """Convert qualitative magnitude to numeric score (0-1)."""
    if pd.isna(magnitude):
        return 0
    magnitude = str(magnitude).lower()
    if 'very_high' in magnitude or 'vhigh' in magnitude:
        return 0.9
    elif 'high' in magnitude:
        return 0.7
    elif 'medium' in magnitude or 'moderate' in magnitude:
        return 0.5
    elif 'low' in magnitude:
        return 0.3
    elif 'very_low' in magnitude or 'vlow' in magnitude:
        return 0.1
    else:
        return 0.5  # Default

# Helper function to convert direction to sign
def direction_to_sign(direction):
    """Convert direction to numeric sign."""
    if pd.isna(direction):
        return 0
    direction = str(direction).lower()
    if 'positive' in direction or 'pos' in direction or 'increase' in direction:
        return 1
    elif 'negative' in direction or 'neg' in direction or 'decrease' in direction:
        return -1
    else:
        return 0

# %%
# Populate matrices from existing impact links
for _, link in impact_with_events.iterrows():
    event_name = link['event_name']
    indicator = link['related_indicator']
    
    if indicator in key_indicators and event_name in association_matrix.index:
        # Calculate impact score
        direction = direction_to_sign(link['impact_direction'])
        magnitude = magnitude_to_score(link['impact_magnitude'])
        impact_score = direction * magnitude
        
        # Update matrices
        association_matrix.loc[event_name, indicator] = impact_score
        direction_matrix.loc[event_name, indicator] = link['impact_direction']
        magnitude_matrix.loc[event_name, indicator] = link['impact_magnitude']
        lag_matrix.loc[event_name, indicator] = link.get('lag_months', 6)  # Default 6 months

print("Association Matrix populated from existing impact links")
print(f"Matrix shape: {association_matrix.shape}")

# Display sample of the matrix
print("\nAssociation Matrix (sample):")
display(association_matrix.head())


#### 3.2 Review Comparable Country Evidence

In [ ]:
# Define comparable countries and their experiences
comparable_countries = {
    'Kenya': {
        'm-pesa_launch': {
            'mobile_money_adoption_increase': 25,  # Percentage points over 5 years
            'lag_months': 12,
            'applicable_events': ['Telebirr launch', 'M-Pesa entry']
        },
        'interoperability_regulation': {
            'digital_payment_increase': 8,  # Percentage points in 18 months
            'lag_months': 6,
            'applicable_events': ['Interoperability regulation enacted']
        }
    },
    'Tanzania': {
        'mobile_money_regulation': {
            'agent_density_increase': 15,  # Agents per 10k adults in 2 years
            'lag_months': 24,
            'applicable_events': ['Regulatory framework updates']
        }
    },
    'Ghana': {
        'mobile_money_interoperability': {
            'transaction_volume_increase': 40,  # Percent increase in 1 year
            'lag_months': 9,
            'applicable_events': ['Interoperability regulation enacted']
        }
    },
    'Rwanda': {
        'digital_id_rollout': {
            'account_ownership_increase': 7,  # Percentage points in 2 years
            'lag_months': 18,
            'applicable_events': ['Fayda Digital ID mass enrollment']
        }
    }
}

print("="*60)
print("COMPARABLE COUNTRY EVIDENCE")
print("="*60)

for country, experiences in comparable_countries.items():
    print(f"\n{country}:")
    for exp_name, exp_data in experiences.items():
        print(f"  • {exp_name}: {exp_data}")


#### 3.3 Estimate Missing Impacts Using Comparable Evidence

In [ ]:
# Define event-indicator relationships based on comparable evidence and expert judgment
event_impact_estimates = {
    # Event: {indicator: (direction, magnitude_score, lag_months, confidence)}
    'Telebirr launch': {
        'ACC_MM_ACCOUNT': ('positive', 0.8, 12, 'high'),
        'USG_DIGITAL_PAYMENT': ('positive', 0.6, 6, 'high'),
        'USG_P2P_VOLUME': ('positive', 0.7, 3, 'medium'),
        'ACC_OWNERSHIP': ('positive', 0.4, 18, 'medium')
    },
    'M-Pesa entry': {
        'ACC_MM_ACCOUNT': ('positive', 0.6, 6, 'high'),
        'USG_DIGITAL_PAYMENT': ('positive', 0.5, 3, 'high'),
        'USG_P2P_VOLUME': ('positive', 0.6, 3, 'medium'),
        'INF_AGENT_DENSITY': ('positive', 0.4, 12, 'medium')
    },
    'Interoperability regulation enacted': {
        'USG_DIGITAL_PAYMENT': ('positive', 0.7, 6, 'medium'),
        'USG_P2P_VOLUME': ('positive', 0.8, 3, 'medium'),
        'ACC_OWNERSHIP': ('positive', 0.3, 12, 'low')
    },
    'Fayda Digital ID mass enrollment': {
        'ACC_OWNERSHIP': ('positive', 0.5, 18, 'medium'),
        'ACC_MM_ACCOUNT': ('positive', 0.4, 12, 'medium'),
        'USG_DIGITAL_PAYMENT': ('positive', 0.3, 24, 'low')
    },
    'National QR code merchant rollout': {
        'USG_DIGITAL_PAYMENT': ('positive', 0.6, 3, 'medium'),
        'USG_P2P_VOLUME': ('positive', 0.4, 6, 'medium')
    },
    'Safaricom market entry': {
        'ACC_MM_ACCOUNT': ('positive', 0.3, 12, 'medium'),
        'INF_AGENT_DENSITY': ('positive', 0.5, 18, 'medium')
    }
}

# %%
# Update matrices with estimated impacts
for event_name, impacts in event_impact_estimates.items():
    if event_name in association_matrix.index:
        for indicator, (direction_str, magnitude_score, lag, confidence) in impacts.items():
            if indicator in association_matrix.columns:
                direction = direction_to_sign(direction_str)
                impact_score = direction * magnitude_score
                
                # Only update if not already populated from impact links
                if pd.isna(association_matrix.loc[event_name, indicator]):
                    association_matrix.loc[event_name, indicator] = impact_score
                    direction_matrix.loc[event_name, indicator] = direction_str
                    magnitude_matrix.loc[event_name, indicator] = f"{magnitude_score:.1f}"
                    lag_matrix.loc[event_name, indicator] = lag

print("Matrices updated with estimated impacts")

#### 3.4 Fill Remaining Gaps with Default/Neutral Values

In [ ]:
# Fill remaining NaN values with 0 (no impact)
association_matrix = association_matrix.fillna(0)
direction_matrix = direction_matrix.fillna('neutral')
magnitude_matrix = magnitude_matrix.fillna('0.0')
lag_matrix = lag_matrix.fillna(0)

print("Association Matrix (complete):")
display(association_matrix)

# %%
# Convert association matrix to numeric for calculations
association_matrix_numeric = association_matrix.astype(float)

# Calculate event strength (sum of absolute impacts)
event_strength = association_matrix_numeric.abs().sum(axis=1).sort_values(ascending=False)
print("\n📊 Event Strength (sum of absolute impacts):")
print(event_strength)

# Calculate indicator sensitivity (sum of absolute impacts)
indicator_sensitivity = association_matrix_numeric.abs().sum(axis=0).sort_values(ascending=False)
print("\n🎯 Indicator Sensitivity (sum of absolute impacts):")
print(indicator_sensitivity)

### 4. Visualize the Association Matrix

In [ ]:
# Create heatmap visualization
plt.figure(figsize=(14, 10))
mask = association_matrix_numeric == 0
sns.heatmap(association_matrix_numeric, 
            cmap='RdBu_r', 
            center=0,
            annot=True, 
            fmt='.2f',
            linewidths=0.5,
            linecolor='gray',
            cbar_kws={'label': 'Impact Score (-1 to +1)'},
            mask=mask)
plt.title('Event-Impact Association Matrix\n(Impact Score: -1 = Strong Negative, +1 = Strong Positive)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Indicators')
plt.ylabel('Events')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../data/processed/event_impact_association_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# %%
# Interactive heatmap with Plotly
fig = px.imshow(association_matrix_numeric,
                labels=dict(x="Indicator", y="Event", color="Impact Score"),
                x=association_matrix_numeric.columns,
                y=association_matrix_numeric.index,
                color_continuous_scale='RdBu',
                color_continuous_midpoint=0,
                title="Event-Impact Association Matrix (Interactive)")
fig.update_layout(height=600)
fig.write_html('../data/processed/event_impact_matrix_interactive.html')
fig.show()

### 5. Test Model Against Historical Data

#### 5.1 Telebirr Launch Impact Validation


In [ ]:
print("="*60)
print("HISTORICAL VALIDATION: TELEBIRR LAUNCH")
print("="*60)

# Get Telebirr launch date
telebirr_date = pd.Timestamp('2021-05-01')

# Get mobile money account data before and after Telebirr
mm_before = observations[
    (observations['indicator_code'] == 'ACC_MM_ACCOUNT') &
    (observations['observation_date'] < telebirr_date)
].sort_values('observation_date')

mm_after = observations[
    (observations['indicator_code'] == 'ACC_MM_ACCOUNT') &
    (observations['observation_date'] > telebirr_date)
].sort_values('observation_date')

# Calculate actual impact
if not mm_before.empty and not mm_after.empty:
    mm_pre = mm_before.iloc[-1]['value_numeric']
    mm_post_1yr = mm_after[mm_after['observation_date'] <= telebirr_date + pd.DateOffset(years=1)]
    mm_post_3yr = mm_after[mm_after['observation_date'] <= telebirr_date + pd.DateOffset(years=3)]
    
    if not mm_post_1yr.empty:
        mm_1yr = mm_post_1yr.iloc[-1]['value_numeric']
    else:
        mm_1yr = None
    
    if not mm_post_3yr.empty:
        mm_3yr = mm_post_3yr.iloc[-1]['value_numeric']
    else:
        mm_3yr = None
    
    print(f"\n📱 Mobile Money Accounts (ACC_MM_ACCOUNT):")
    print(f"   Before Telebirr launch: {mm_pre}%")
    if mm_1yr:
        print(f"   ~1 year after launch: {mm_1yr}%")
        print(f"   Change: +{mm_1yr - mm_pre:.2f}pp")
    if mm_3yr:
        print(f"   ~3 years after launch: {mm_3yr}%")
        print(f"   Change: +{mm_3yr - mm_pre:.2f}pp")
    
    # Compare with model prediction
    model_impact = association_matrix_numeric.loc['Telebirr launch', 'ACC_MM_ACCOUNT']
    print(f"\n📊 Model Prediction for ACC_MM_ACCOUNT:")
    print(f"   Impact score: {model_impact:.2f}")
    print(f"   Direction: {direction_matrix.loc['Telebirr launch', 'ACC_MM_ACCOUNT']}")
    print(f"   Magnitude: {magnitude_matrix.loc['Telebirr launch', 'ACC_MM_ACCOUNT']}")
    print(f"   Lag: {lag_matrix.loc['Telebirr launch', 'ACC_MM_ACCOUNT']} months")

#### 5.2 M-Pesa Entry Impact Validation

In [ ]:
print("\n" + "="*60)
print("HISTORICAL VALIDATION: M-PESA ENTRY")
print("="*60)

# Get M-Pesa entry date
mpesa_date = pd.Timestamp('2023-08-01')

# Get data before and after M-Pesa entry
# Since we don't have 2025 data yet, we'll use the latest available
mpesa_before = observations[
    (observations['observation_date'] < mpesa_date)
].sort_values('observation_date')

mpesa_after = observations[
    (observations['observation_date'] > mpesa_date)
].sort_values('observation_date')

print(f"\n📅 Data Availability:")
print(f"   Latest data before M-Pesa: {mpesa_before['observation_date'].max().strftime('%Y-%m') if not mpesa_before.empty else 'N/A'}")
print(f"   Earliest data after M-Pesa: {mpesa_after['observation_date'].min().strftime('%Y-%m') if not mpesa_after.empty else 'N/A'}")

# Get specific indicators
for indicator in ['ACC_MM_ACCOUNT', 'USG_DIGITAL_PAYMENT']:
    ind_before = mpesa_before[mpesa_before['indicator_code'] == indicator]
    ind_after = mpesa_after[mpesa_after['indicator_code'] == indicator]
    
    if not ind_before.empty and not ind_after.empty:
        before_val = ind_before.iloc[-1]['value_numeric']
        after_val = ind_after.iloc[0]['value_numeric']
        change = after_val - before_val
        
        print(f"\n📈 {indicator}:")
        print(f"   Before M-Pesa: {before_val}%")
        print(f"   After M-Pesa: {after_val}%")
        print(f"   Change: {change:+.2f}pp")
        
        # Compare with model
        model_impact = association_matrix_numeric.loc['M-Pesa entry', indicator]
        print(f"   Model impact score: {model_impact:.2f}")


#### 5.3 Statistical Analysis of Event Impacts

In [ ]:
# Create event impact timeline for visualization
event_impacts = []

for event_name in association_matrix_numeric.index:
    event_date = events[events['event_name'] == event_name]['event_date']
    if not event_date.empty:
        event_date = event_date.iloc[0]
        
        # Get indicators affected by this event
        affected_indicators = association_matrix_numeric.columns[
            association_matrix_numeric.loc[event_name].abs() > 0
        ]
        
        for indicator in affected_indicators:
            impact_score = association_matrix_numeric.loc[event_name, indicator]
            if impact_score != 0:
                event_impacts.append({
                    'event': event_name,
                    'event_date': event_date,
                    'indicator': indicator,
                    'impact_score': impact_score,
                    'direction': direction_matrix.loc[event_name, indicator],
                    'magnitude': magnitude_matrix.loc[event_name, indicator],
                    'lag_months': lag_matrix.loc[event_name, indicator]
                })

event_impacts_df = pd.DataFrame(event_impacts)
print(f"\nTotal event-impact relationships modeled: {len(event_impacts_df)}")

# %%
# Visualize event impacts over time
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Impact distribution by event category
event_categories = events.set_index('event_name')['event_category']
event_impacts_df['event_category'] = event_impacts_df['event'].map(event_categories)

if 'event_category' in event_impacts_df.columns:
    category_impact = event_impacts_df.groupby('event_category')['impact_score'].agg(['mean', 'count', 'std'])
    axes[0,0].bar(category_impact.index, category_impact['mean'].abs(), 
                  color=['green', 'blue', 'orange', 'red'][:len(category_impact)])
    axes[0,0].set_title('Average Impact Strength by Event Category', fontweight='bold')
    axes[0,0].set_ylabel('Average |Impact Score|')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Add count labels
    for i, (idx, row) in enumerate(category_impact.iterrows()):
        axes[0,0].text(i, row['mean'].abs() + 0.02, f"n={int(row['count'])}", 
                      ha='center', fontsize=9)

# 2. Impact distribution by pillar
pillar_mapping = {
    'ACC': 'Access',
    'USG': 'Usage',
    'INF': 'Infrastructure',
    'ENB': 'Enablers'
}
event_impacts_df['pillar'] = event_impacts_df['indicator'].apply(
    lambda x: x.split('_')[0] if '_' in str(x) else 'OTHER'
)
event_impacts_df['pillar_label'] = event_impacts_df['pillar'].map(pillar_mapping).fillna('Other')

pillar_impact = event_impacts_df.groupby('pillar_label')['impact_score'].agg(['mean', 'count'])
axes[0,1].bar(pillar_impact.index, pillar_impact['mean'].abs(), 
              color=['darkblue', 'darkgreen', 'darkred', 'darkorange'][:len(pillar_impact)])
axes[0,1].set_title('Average Impact Strength by Pillar', fontweight='bold')
axes[0,1].set_ylabel('Average |Impact Score|')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Lag distribution
axes[1,0].hist(event_impacts_df['lag_months'].dropna(), bins=10, 
               color='purple', alpha=0.7, edgecolor='black')
axes[1,0].set_title('Distribution of Impact Lag Times', fontweight='bold')
axes[1,0].set_xlabel('Lag (months)')
axes[1,0].set_ylabel('Count')
axes[1,0].axvline(event_impacts_df['lag_months'].median(), color='red', 
                 linestyle='--', label=f'Median: {event_impacts_df["lag_months"].median():.0f} months')
axes[1,0].legend()

# 4. Impact score distribution
axes[1,1].hist(event_impacts_df['impact_score'], bins=20, 
               color='teal', alpha=0.7, edgecolor='black')
axes[1,1].set_title('Distribution of Impact Scores', fontweight='bold')
axes[1,1].set_xlabel('Impact Score')
axes[1,1].set_ylabel('Count')
axes[1,1].axvline(0, color='red', linestyle='-', alpha=0.5)

plt.tight_layout()
plt.savefig('../data/processed/event_impact_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

### 6. Refine Impact Estimates

In [ ]:
print("="*60)
print("REFINING IMPACT ESTIMATES")
print("="*60)

# Based on historical validation, adjust estimates
adjustments = []

# 1. Telebirr impact adjustment based on actual data
if 'mm_pre' in locals() and 'mm_3yr' in locals():
    actual_telebirr_impact = (mm_3yr - mm_pre) / 100  # Convert pp to proportion
    model_telebirr_impact = association_matrix_numeric.loc['Telebirr launch', 'ACC_MM_ACCOUNT']
    
    if abs(actual_telebirr_impact - model_telebirr_impact) > 0.1:
        adjustments.append({
            'event': 'Telebirr launch',
            'indicator': 'ACC_MM_ACCOUNT',
            'old_score': model_telebirr_impact,
            'new_score': actual_telebirr_impact,
            'reason': f'Actual growth: +{mm_3yr - mm_pre:.1f}pp over 3 years'
        })
        # Update the matrix
        association_matrix_numeric.loc['Telebirr launch', 'ACC_MM_ACCOUNT'] = actual_telebirr_impact

# 2. Adjust based on comparable country evidence
for event_name in ['Interoperability regulation enacted', 'Fayda Digital ID mass enrollment']:
    if event_name in association_matrix_numeric.index:
        # Increase impact on digital payments based on Kenya evidence
        if 'USG_DIGITAL_PAYMENT' in association_matrix_numeric.columns:
            current_impact = association_matrix_numeric.loc[event_name, 'USG_DIGITAL_PAYMENT']
            if abs(current_impact) < 0.5:  # If impact is relatively low
                new_impact = min(0.7, current_impact + 0.2)  # Increase by 0.2, cap at 0.7
                adjustments.append({
                    'event': event_name,
                    'indicator': 'USG_DIGITAL_PAYMENT',
                    'old_score': current_impact,
                    'new_score': new_impact,
                    'reason': 'Based on Kenya interoperability evidence (+8-12pp impact)'
                })
                association_matrix_numeric.loc[event_name, 'USG_DIGITAL_PAYMENT'] = new_impact

print(f"\nMade {len(adjustments)} adjustments based on validation:")

if adjustments:
    adjustments_df = pd.DataFrame(adjustments)
    display(adjustments_df)
else:
    print("No significant adjustments needed based on current validation.")

### 7. Document Confidence Levels

In [ ]:
# Create confidence matrix
confidence_matrix = pd.DataFrame(index=association_matrix_numeric.index, 
                                 columns=association_matrix_numeric.columns)

# Assign confidence based on:
# 1. Existence of historical data for validation
# 2. Comparable country evidence
# 3. Expert judgment

confidence_criteria = {
    'high': [
        ('Telebirr launch', 'ACC_MM_ACCOUNT'),
        ('M-Pesa entry', 'ACC_MM_ACCOUNT'),
        ('Telebirr launch', 'USG_DIGITAL_PAYMENT')
    ],
    'medium': [
        ('Interoperability regulation enacted', 'USG_DIGITAL_PAYMENT'),
        ('Fayda Digital ID mass enrollment', 'ACC_OWNERSHIP'),
        ('National QR code merchant rollout', 'USG_DIGITAL_PAYMENT')
    ],
    'low': [
        ('Fayda Digital ID mass enrollment', 'USG_DIGITAL_PAYMENT'),
        ('Interoperability regulation enacted', 'ACC_OWNERSHIP')
    ]
}

# Initialize all as medium confidence
confidence_matrix[:] = 'medium'

# Apply specific confidence levels
for confidence_level, pairs in confidence_criteria.items():
    for event, indicator in pairs:
        if event in confidence_matrix.index and indicator in confidence_matrix.columns:
            confidence_matrix.loc[event, indicator] = confidence_level

print("Confidence Matrix (sample):")
display(confidence_matrix.head())

### 8. Create Impact Model Functions

In [ ]:
def calculate_event_impact(event_name, indicator, months_after_event):
    """
    Calculate the impact of an event on an indicator at a specific time.
    
    Parameters:
    -----------
    event_name : str
        Name of the event
    indicator : str
        Indicator code
    months_after_event : int
        Number of months after the event
    
    Returns:
    --------
    impact : float
        Impact score (0 to 1 for positive, -1 to 0 for negative)
    """
    if event_name not in association_matrix_numeric.index or indicator not in association_matrix_numeric.columns:
        return 0.0
    
    base_impact = association_matrix_numeric.loc[event_name, indicator]
    lag_months = lag_matrix.loc[event_name, indicator]
    
    if pd.isna(lag_months) or base_impact == 0:
        return 0.0
    
    # Simple sigmoid function for impact buildup over time
    if months_after_event <= 0:
        return 0.0
    elif months_after_event < lag_months:
        # Building up to full impact
        buildup_factor = months_after_event / lag_months
        return base_impact * buildup_factor
    else:
        # Full impact achieved
        return base_impact

def calculate_combined_impact(events, indicator, date):
    """
    Calculate combined impact of multiple events on an indicator at a specific date.
    
    Parameters:
    -----------
    events : list of tuples
        List of (event_name, event_date) tuples
    indicator : str
        Indicator code
    date : datetime
        Date for which to calculate impact
    
    Returns:
    --------
    total_impact : float
        Sum of individual event impacts
    """
    total_impact = 0.0
    
    for event_name, event_date in events:
        if pd.isna(event_date):
            continue
            
        months_after = (date.year - event_date.year) * 12 + (date.month - event_date.month)
        impact = calculate_event_impact(event_name, indicator, months_after)
        total_impact += impact
    
    return total_impact

def forecast_with_events(base_forecast, events, indicator, forecast_dates):
    """
    Generate forecast incorporating event impacts.
    
    Parameters:
    -----------
    base_forecast : pd.Series
        Baseline forecast without events
    events : list of tuples
        List of (event_name, event_date) tuples
    indicator : str
        Indicator code
    forecast_dates : pd.DatetimeIndex
        Dates to forecast
    
    Returns:
    --------
    event_adjusted_forecast : pd.Series
        Forecast with event impacts incorporated
    """
    event_impacts = pd.Series(index=forecast_dates, dtype=float)
    
    for date in forecast_dates:
        impact = calculate_combined_impact(events, indicator, date)
        event_impacts[date] = impact
    
    # Convert impact scores to percentage points
    # Assuming 1.0 impact score = 10 percentage points
    impact_pp = event_impacts * 10
    
    # Add to base forecast
    adjusted_forecast = base_forecast + impact_pp
    
    return adjusted_forecast


### 9. Test the Impact Model

In [ ]:
print("="*60)
print("MODEL TESTING WITH HISTORICAL DATA")
print("="*60)

# Test: What would our model have predicted for Telebirr impact?
test_event = [('Telebirr launch', pd.Timestamp('2021-05-01'))]
test_dates = pd.date_range(start='2021-05-01', end='2024-12-01', freq='MS')

# Create a simple baseline (linear growth from pre-Telebirr level)
if 'mm_pre' in locals():
    baseline = pd.Series(index=test_dates, dtype=float)
    # Simple linear interpolation
    for i, date in enumerate(test_dates):
        months_from_start = i
        baseline[date] = mm_pre + (months_from_start / 36) * (mm_3yr - mm_pre)  # Over 3 years
    
    # Apply event impact model
    model_forecast = forecast_with_events(baseline, test_event, 'ACC_MM_ACCOUNT', test_dates)
    
    # Get actual data points
    actual_dates = []
    actual_values = []
    for _, row in observations[observations['indicator_code'] == 'ACC_MM_ACCOUNT'].iterrows():
        actual_dates.append(row['observation_date'])
        actual_values.append(row['value_numeric'])
    
    # Plot comparison
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot baseline and model forecast
    ax.plot(baseline.index, baseline.values, 'b--', label='Baseline (no Telebirr)', alpha=0.7)
    ax.plot(model_forecast.index, model_forecast.values, 'g-', label='Model with Telebirr impact', linewidth=2)
    
    # Plot actual data points
    ax.scatter(actual_dates, actual_values, color='red', s=100, label='Actual data', zorder=5)
    
    ax.set_title('Model Test: Telebirr Impact on Mobile Money Accounts', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Mobile Money Accounts (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../data/processed/model_test_telebirr.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Calculate model error
    if actual_values and len(actual_values) >= 2:
        # Compare last actual with model prediction at that time
        last_actual_date = actual_dates[-1]
        last_actual_value = actual_values[-1]
        
        if last_actual_date in model_forecast.index:
            model_prediction = model_forecast[last_actual_date]
            error = abs(model_prediction - last_actual_value)
            error_pct = (error / last_actual_value) * 100
            
            print(f"\n📊 Model Performance:")
            print(f"   Actual (2024): {last_actual_value:.2f}%")
            print(f"   Model prediction: {model_prediction:.2f}%")
            print(f"   Absolute error: {error:.2f}pp")
            print(f"   Relative error: {error_pct:.1f}%")

### 10. Document Methodology and Assumptions

In [ ]:
print("="*60)
print("METHODOLOGY DOCUMENTATION")
print("="*60)

methodology = {
    'impact_scoring': {
        'description': 'Impact scores range from -1 (strong negative) to +1 (strong positive)',
        'magnitude_mapping': {
            'very_high': 0.9,
            'high': 0.7,
            'medium': 0.5,
            'low': 0.3,
            'very_low': 0.1
        },
        'direction_mapping': {
            'positive': 1,
            'negative': -1,
            'neutral': 0
        }
    },
    'temporal_dynamics': {
        'impact_buildup': 'Linear buildup to full impact over lag period',
        'lag_periods': 'Based on event type: product launches (3-12 months), policies (6-24 months)',
        'persistence': 'Assumed permanent impact once fully realized'
    },
    'combined_effects': {
        'assumption': 'Additive combination of individual event impacts',
        'limitation': 'Does not model interaction/synergy effects between events',
        'note': 'Conservative approach to avoid overestimation'
    },
    'data_sources': {
        'historical_validation': 'Used actual Ethiopian data where available (Telebirr, M-Pesa)',
        'comparable_evidence': 'Kenya (M-Pesa, interoperability), Rwanda (digital ID), Ghana (interoperability)',
        'expert_judgment': 'Applied for events without direct precedents in Ethiopia'
    },
    'uncertainty_handling': {
        'confidence_levels': 'High/Medium/Low based on evidence quality',
        'sensitivity': 'Impact scores represent central estimates',
        'recommendation': 'Use ±30% range for low-confidence estimates in scenarios'
    }
}

print("\n📋 Methodology Summary:")
for category, details in methodology.items():
    print(f"\n{category.upper().replace('_', ' ')}:")
    if isinstance(details, dict):
        for key, value in details.items():
            print(f"  • {key}: {value}")
    else:
        print(f"  {details}")

# Save methodology documentation
import json
with open('../data/processed/impact_modeling_methodology.json', 'w') as f:
    json.dump(methodology, f, indent=2)

print(f"\n✅ Methodology saved to '../data/processed/impact_modeling_methodology.json'")



### 11. Limitations and Uncertainties

In [ ]:
print("="*60)
print("MODEL LIMITATIONS AND UNCERTAINTIES")
print("="*60)

limitations = [
    {
        'limitation': 'Limited Ethiopian historical data',
        'description': 'Few events with pre/post data for validation in Ethiopia',
        'mitigation': 'Relied on comparable country evidence and expert judgment'
    },
    {
        'limitation': 'Additive impact assumption',
        'description': 'Assumes events have independent, additive effects (no interactions)',
        'mitigation': 'Conservative estimates; future enhancement: interaction terms'
    },
    {
        'limitation': 'Simplified temporal dynamics',
        'description': 'Linear buildup of impacts, instant persistence after lag',
        'mitigation': 'Adequate for annual forecasts; refine for monthly precision'
    },
    {
        'limitation': 'Qualitative to quantitative conversion',
        'description': 'Converting qualitative impact estimates to numeric scores involves judgment',
        'mitigation': 'Used consistent mapping scheme; document assumptions'
    },
    {
        'limitation': 'Static impact magnitudes',
        'description': 'Impact magnitudes don\'t vary with economic conditions or saturation',
        'mitigation': 'Scenario analysis will address different conditions'
    }
]

print("\n⚠️  Key Limitations:")
for i, limit in enumerate(limitations, 1):
    print(f"\n{i}. {limit['limitation']}")
    print(f"   Description: {limit['description']}")
    print(f"   Mitigation: {limit['mitigation']}")

# Save limitations
limitations_df = pd.DataFrame(limitations)
limitations_df.to_csv('../data/processed/impact_modeling_limitations.csv', index=False)

### 12. Task 3 Completion Summary

In [ ]:
print("="*60)
print("TASK 3 COMPLETION SUMMARY")
print("="*60)

# Save all matrices
output_files = []

# Save association matrix
association_matrix_path = '../data/processed/event_indicator_association_matrix.csv'
association_matrix_numeric.to_csv(association_matrix_path)
output_files.append(association_matrix_path)
print(f"✓ Association matrix saved: {association_matrix_path}")

# Save direction matrix
direction_matrix_path = '../data/processed/event_impact_directions.csv'
direction_matrix.to_csv(direction_matrix_path)
output_files.append(direction_matrix_path)

# Save magnitude matrix
magnitude_matrix_path = '../data/processed/event_impact_magnitudes.csv'
magnitude_matrix.to_csv(magnitude_matrix_path)
output_files.append(magnitude_matrix_path)

# Save lag matrix
lag_matrix_path = '../data/processed/event_impact_lags.csv'
lag_matrix.to_csv(lag_matrix_path)
output_files.append(lag_matrix_path)

# Save confidence matrix
confidence_matrix_path = '../data/processed/event_impact_confidence.csv'
confidence_matrix.to_csv(confidence_matrix_path)
output_files.append(confidence_matrix_path)

# Save all event impacts
event_impacts_df_path = '../data/processed/all_event_impacts.csv'
event_impacts_df.to_csv(event_impacts_df_path, index=False)
output_files.append(event_impacts_df_path)

print(f"\n📊 Model Statistics:")
print(f"   Events modeled: {len(association_matrix_numeric)}")
print(f"   Indicators covered: {len(association_matrix_numeric.columns)}")
print(f"   Total event-indicator relationships: {len(event_impacts_df)}")
print(f"   Average impact strength: {event_impacts_df['impact_score'].abs().mean():.3f}")
print(f"   Median lag time: {event_impacts_df['lag_months'].median():.0f} months")

print(f"\n✅ Output files generated: {len(output_files)} files")
for file in output_files[:3]:  # Show first 3
    print(f"  • {file.split('/')[-1]}")
if len(output_files) > 3:
    print(f"  • ... and {len(output_files) - 3} more")

print("\n🎯 Key Events by Impact Strength:")
top_events = event_strength.head(5)
for event, strength in top_events.items():
    print(f"  • {event}: {strength:.2f}")

print("\n📈 Key Indicators by Sensitivity:")
top_indicators = indicator_sensitivity.head(5)
for indicator, sensitivity in top_indicators.items():
    print(f"  • {indicator}: {sensitivity:.2f}")